In [1]:
import pandas as pd

df = pd.read_csv('./data/HS116_members.csv')
df = df.loc[pd.notnull(df.nominate_dim1)].reset_index(drop=True)
df['first_middle_name'] = df['bioname'].apply(lambda x: x.split(', ')[1])
df['first_name'] = df['first_middle_name'].apply(lambda x: x.split(' ')[0] + ' ' if ' ' in x else x + ' ')
df['middle_name'] = df['first_middle_name'].apply(lambda x: x.split(' ')[1][0] + '. ' if ' ' in x else '')
df['last_name'] = df['bioname'].apply(lambda x: x.split(', ')[0].lower().capitalize().strip())
df['fullname'] = df['first_name'] + df['middle_name'] + df['last_name']

In [12]:
import asyncio
import os
from openai import AsyncOpenAI
from typing import List, Dict
import random

# Load your API key
client = AsyncOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://silk.wannaexpresso.com:3090/v1",
)

# Concurrency control (adjust this if you run into rate limits)
MAX_CONCURRENT_REQUESTS = 10
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

# Policy queries
policy_queries = {
    "Abortion": "On a scale from 1 to 7, where 1 means strongly oppose and 7 means strongly favor, what is your opinion on making abortion always legal?",
    "Border Wall": "On a scale from 1 to 7, where 1 means strongly oppose and 7 means strongly favor, what is your opinion on building a wall on the U.S. border with Mexico?",
    "Transgender Bathroom": "On a scale from 1 to 7, where 1 means strongly oppose and 7 means strongly favor, what is your opinion on allowing transgender people to use public bathrooms matching their gender identity?",
    "Ukraine Military Aid": "On a scale from 1 to 7, where 1 means strongly oppose and 7 means strongly favor, what is your opinion on the U.S. giving weapons to help Ukraine fight Russia?",
    "Israel Military Aid": "On a scale from 1 to 7, where 1 means strongly oppose and 7 means strongly favor, what is your opinion on the U.S. giving military assistance to Israel?",
    "Gaza Humanitarian Aid": "On a scale from 1 to 7, where 1 means strongly oppose and 7 means strongly favor, what is your opinion on the U.S. providing humanitarian assistance to Palestinians in the Gaza Strip?"
}

def build_prompt(name: str, domain: str) -> str:
    return (
        f"You are {name}, a U.S. politician. "
        f"Write a realistic one-sentence statement that {name} might give about {domain}."
    )

async def query_policy_statement(name: str, domain: str) -> Dict:
    prompt = build_prompt(name, domain)
    async with semaphore:
        try:
            response = await client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7,
                max_tokens=150
            )
            content = response.choices[0].message.content.strip()
            return {"name": name, "domain": domain, "response": content}
        except Exception as e:
            return {"name": name, "domain": domain, "response": f"ERROR: {str(e)}"}

async def main():
    tasks: List[asyncio.Task] = []

    for domain, _ in policy_queries.items():
        for name in df['fullname'].sample(20, random_state=random.randint(0, 42)).tolist():
            tasks.append(asyncio.create_task(query_policy_statement(name, domain)))

    results = await asyncio.gather(*tasks)

    # Print or process results
    for r in results:
        print(f"{r['name']} on {r['domain']}:\n{r['response']}\n")

    # Optionally write to CSV
    pd.DataFrame(results).to_csv("policy_statements_parallel.csv", index=False)

await main()


Raúl M. Grijalva on Abortion:
"As a staunch advocate for reproductive rights, I firmly believe that every individual should have the autonomy to make their own healthcare decisions, including the right to access safe and legal abortion services."

Jaime Herrera beutler on Abortion:
"As a pro-life advocate, I believe in protecting the sanctity of life while also supporting policies that provide women with the resources and support they need during challenging times."

Cheri Bustos on Abortion:
"As a strong advocate for women's rights, I believe that access to safe and legal abortion is a fundamental aspect of healthcare and a personal decision that should be made between a woman and her doctor, free from government interference."

Richard Blumenthal on Abortion:
"As a staunch advocate for women's rights, I believe that access to safe and legal abortion is a fundamental aspect of healthcare that must be protected and upheld for all women across our nation."

C. A. Ruppersberger on Aborti